In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Oct 31 17:15:29 2023
@author: nesteele
"""

import pandas as pd
import matplotlib.pyplot as plt
from StackedLinePlots import StackedLinePlot

class LowHighFlowYear(StackedLinePlot):

    def __init__(self, csv_file, name_of_date_column, name_of_Q_column):
        super().__init__(csv_file, name_of_date_column, name_of_Q_column)
        self._yearly_stats_df = None
        self._lowflow_years = None
        self._highflow_years = None
        self._lowflow_stats = None
        self._highflow_stats = None

    @property
    def getYearlyStatsDF(self):
        return self._yearly_stats_df

    @property
    def getLowFlowYears(self):
        return self._lowflow_years

    def calculateLowHighFlowStatistics(self, n_years):
        yearly_stats = {i: self._pivot_table.loc[:, i].describe() for i in self._pivot_table.columns}
        self._yearly_stats_df = pd.DataFrame([yearly_stats])
        yearly_means = {i: yearly_stats[i].T['mean'] for i in self._pivot_table.columns}
        yearly_means_df = pd.DataFrame([yearly_means]).T.reset_index()
        yearly_means_df.columns = ['Year', 'Mean']
        yearly_means_df = yearly_means_df.sort_values(by=['Mean'], ascending=True).set_index('Year')
        lowflow_years_means_df = yearly_means_df.iloc[:n_years, :]
        highflow_years_means_df = yearly_means_df.iloc[-n_years:, :]
        self._lowflow_years = sorted(list(lowflow_years_means_df.index)[-n_years:])
        self._highflow_years = sorted(list(highflow_years_means_df.index)[:n_years])

        self._lowflow_pivot_table = self._pivot_table.loc[:, self._lowflow_years]
        lowflow_df = self._df[self._df['Year'].isin(self._lowflow_years)]
        self._lowflow_stats = lowflow_df.groupby("month-day")[self._name_of_Q_column].agg(['mean', 'median', 'std', ("q25", lambda x: x.quantile(0.25)), ("q75", lambda y: y.quantile(0.75))])
        self._lowflow_mean = self._lowflow_stats.iloc[:, 0]
        self._lowflow_median = self._lowflow_stats.iloc[:, 1]
        self._lowflow_percentile25 = self._lowflow_stats.iloc[:, 3]
        self._lowflow_percentile75 = self._lowflow_stats.iloc[:, 4]
        self._lowflow_lower_bound_percentile25 = self._lowflow_mean - self._lowflow_percentile25
        self._lowflow_upper_bound_percentile75 = self._lowflow_mean + self._lowflow_percentile75

        self._highflow_pivot_table = self._pivot_table.loc[:, self._highflow_years]
        highflow_df = self._df[self._df['Year'].isin(self._highflow_years)]
        self._highflow_stats = highflow_df.groupby("month-day")[self._name_of_Q_column].agg(['mean', 'median', 'std', ("q25", lambda x: x.quantile(0.25)), ("q75", lambda y: y.quantile(0.75))])
        self._highflow_mean = self._highflow_stats.iloc[:, 0]
        self._highflow_median = self._highflow_stats.iloc[:, 1]
        self._highflow_percentile25 = self._highflow_stats.iloc[:, 3]
        self._highflow_percentile75 = self._highflow_stats.iloc[:, 4]
        self._highflow_lower_bound_percentile25 = self._highflow_mean - self._highflow_percentile25
        self._highflow_upper_bound_percentile75 = self._highflow_mean + self._highflow_percentile75

    def plotLowHighStackedLinePlot(self,
                                    forced_x_positions=[1, 32, 60, 91, 121, 152, 182, 213, 244, 274, 305, 336],
                                    forced_x_labels=['01-01', '02-01', '03-01', '04-01', '05-01', '06-01', '07-01', '08-01', '09-01', '10-01', '11-01', '12-01'],
                                    title=None,
                                    quartile_shading=True,
                                    quartile_shading_alpha=0.4,
                                    y_upper_lim="Auto",
                                    legend='upper right',
                                    legend_ncol=1):
        self._forced_x_positions = forced_x_positions
        self._forced_x_labels = forced_x_labels

        fig, ax = plt.subplots(figsize=(9, 7))
        self._lowflow_mean.plot(ax=ax, label="Mean", linestyle='--', color='black', linewidth=2.0, zorder=3)
        self._lowflow_median.plot(ax=ax, label="Median", linestyle='--', color='red', linewidth=2.0, zorder=3)

        valid_years = [year for year in self._lowflow_pivot_table.columns if self._lowflow_pivot_table[year].count() > 300]
        filtered_lowflow_pivot_table = self._lowflow_pivot_table.loc[:, valid_years]
        filtered_lowflow_pivot_table.plot(ax=ax, alpha=1, zorder=2, linewidth=1.0, color=self._colors)

        if self._forced_x_positions is not None and self._forced_x_labels is not None:
            ax.set_xticks(self._forced_x_positions)
            xlim_min = self._forced_x_positions[0]
            xlim_max = self._forced_x_positions[-1]
            ax.set_xticklabels(self._forced_x_labels, rotation=45)
            ax.set_xlim([xlim_min, xlim_max])
            ax.set_ylim([0, y_upper_lim])

            plt.grid(color='green', linestyle=":", linewidth=0.5)
            plt.xlabel('Month-Day')
            plt.ylabel('Q (cfs)')
            plt.title("{} - low water years".format(title))
            plt.legend(loc=legend, ncol=legend_ncol)
            plt.show()

        fig, ax = plt.subplots(figsize=(9, 7))
        self._highflow_mean.plot(ax=ax, label="Mean", linestyle='--', color='black', linewidth=2.0, zorder=3)
        self._highflow_median.plot(ax=ax, label="Median", linestyle='--', color='red', linewidth=2.0, zorder=3)

        valid_years = [year for year in self._highflow_pivot_table.columns if self._highflow_pivot_table[year].count() > 300]
        filtered_highflow_pivot_table = self._highflow_pivot_table.loc[:, valid_years]
        filtered_highflow_pivot_table.plot(ax=ax, alpha=1, zorder=2, linewidth=1.0)

        if self._forced_x_positions is not None and self._forced_x
        if self._forced_x_positions is not None and self._forced_x_labels is not None:
            ax.set_xticks(self._forced_x_positions)
            xlim_min = self._forced_x_positions[0]
            xlim_max = self._forced_x_positions[-1]
            ax.set_xticklabels(self._forced_x_labels, rotation=45)
            ax.set_xlim([xlim_min, xlim_max])
            ax.set_ylim([0, y_upper_lim])

            plt.grid(color='green', linestyle=":", linewidth=0.5)
            plt.xlabel('Month-Day')
            plt.ylabel('Q (cfs)')
            plt.title("{} - high water years".format(title))
            plt.legend(loc=legend, ncol=legend_ncol)
            plt.show()


if __name__ == "__main__":
    print("This module creates customized StackedLinePlots.")
    # Uncomment and provide the necessary arguments to run the script
    # csv_file_path = 'path_to_your_csv_file.csv'
    # date_column_name = 'your_date_column_name'
    # Q_column_name = 'your_Q_column_name'
    # n_years_to_plot = 5
    # low_high_flow_year = LowHighFlowYear(csv_file_path, date_column_name, Q_column_name)
    # low_high_flow_year.calculateLowHighFlowStatistics(n_years_to_plot)
    # low_high_flow_year.plotLowHighStackedLinePlot(title='Your Title')

SyntaxError: expected ':' (2600073599.py, line 105)